# FinBERT Profiling: Baseline 

This notebook is intentionally **thin**: it reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.




In [1]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data
import finbert.utils as tools


%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

✓ Helper utilities loaded


In [2]:
wandb.init(
    entity="si2449-columbia-university",
    project="finbert-experiments",
    name="training-baseline-profiled",
    group="baseline"
)

wandb: Currently logged in as: shriya-mahakala (aqlab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
cl_path = project_dir/'models'/'teacher'
cl_data_path = project_dir/'data'/'sentiment_data'

In [4]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

config.profile_train_steps = 20

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from contextlib import nullcontext
from typing import Any

import torch.nn.functional as F
from sklearn.metrics import f1_score


def timed_eval(
    *, finbert: FinBert, model: torch.nn.Module, examples, use_amp: bool
) -> tuple[pd.DataFrame, dict[str, Any]]:
    """Evaluation loop with optional CUDA autocast + timing (kept small for notebook use)."""
    loader = finbert.get_loader(examples, phase="eval")
    device = finbert.device

    model.eval()
    preds: list[np.ndarray] = []
    labels: list[int] = []

    if device.type == "cuda":
        torch.cuda.synchronize(device)

    start = time.perf_counter()
    with torch.no_grad():
        for batch in loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, _agree_ids = batch

            logits = model(input_ids, attention_mask, token_type_ids)[0]

            preds.extend(logits.detach().cpu().numpy())
            labels.extend(label_ids.detach().cpu().numpy().tolist())

    if device.type == "cuda":
        torch.cuda.synchronize(device)

    wall_s = time.perf_counter() - start
    n = len(labels)

    results_df = pd.DataFrame({"predictions": preds, "labels": labels})

    timing = {
        "eval_wall_s": float(wall_s),
        "eval_num_samples": int(n),
        "eval_samples_per_s": float(n / wall_s) if wall_s > 0 else float("inf"),
    }
    return results_df, timing

In [6]:
finbert = ProfiledFinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [7]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/14/2025 20:56:14 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [8]:
train_data = finbert.get_data('train')

In [9]:
model = finbert.create_the_model()

In [10]:
freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

In [11]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start



12/14/2025 20:56:16 - INFO - finbert.utils -   *** Example ***
12/14/2025 20:56:16 - INFO - finbert.utils -   guid: train-1
12/14/2025 20:56:16 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/14/2025 20:56:16 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:56:16 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:56:16 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/

12/14/2025 20:56:16 - INFO - finbert.finbert -   ***** Loading data *****
12/14/2025 20:56:16 - INFO - finbert.finbert -     Num examples = 3488
12/14/2025 20:56:16 - INFO - finbert.finbert -     Batch size = 32
12/14/2025 20:56:16 - INFO - finbert.finbert -     Num steps = 48
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_profile.py:138: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.use_amp)


\n================================================================================
Starting Profiled Training
Device: cuda
Profiling activities: [<ProfilerActivity.CPU: 0>, <ProfilerActivity.CUDA: 2>]
================================================================================\n


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_profile.py:191: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config.use_amp):
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_profile.py:196: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config.use_amp):
Epoch:   0%|          | 0/4 [00:02<?, ?it/s]


\n================================================================================
Profiling complete for first epoch (20 steps)
Continuing full training without profiling...
================================================================================\n
\n================================================================================
PROFILING RESULTS - Training
================================================================================\n
\nBy CPU Time:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
----------------------

...

Iteration: 100%|██████████| 109/109 [00:23<00:00,  4.69it/s]
12/14/2025 20:57:24 - INFO - finbert.utils -   *** Example ***
12/14/2025 20:57:24 - INFO - finbert.utils -   guid: validation-1
12/14/2025 20:57:24 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/14/2025 20:57:24 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:24 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:24 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:24 - INFO - finbert.utils -   label: neutral (id = 2)
12/14/2025 20:57:24 

Validation losses: [0.9712502910540655, 0.6004666571433728, 0.49841671035839963]


Iteration: 100%|██████████| 109/109 [00:26<00:00,  4.08it/s]
12/14/2025 20:57:52 - INFO - finbert.utils -   *** Example ***
12/14/2025 20:57:52 - INFO - finbert.utils -   guid: validation-1
12/14/2025 20:57:52 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/14/2025 20:57:52 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:52 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:52 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:52 - INFO - finbert.utils -   label: neutral (id = 2)
12/14/2025 20:57:52 

Validation losses: [0.9712502910540655, 0.6004666571433728, 0.49841671035839963, 0.482469033736449]


Epoch: 100%|██████████| 4/4 [01:26<00:00, 21.68s/it]


In [12]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)


eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)


12/14/2025 20:57:56 - INFO - finbert.utils -   *** Example ***
12/14/2025 20:57:56 - INFO - finbert.utils -   guid: test-1
12/14/2025 20:57:56 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/14/2025 20:57:56 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:56 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:56 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/14/2025 20:57:56 - IN

12/14/2025 20:57:56 - INFO - finbert.finbert -   ***** Loading data *****
12/14/2025 20:57:56 - INFO - finbert.finbert -     Num examples = 970
12/14/2025 20:57:56 - INFO - finbert.finbert -     Batch size = 32
12/14/2025 20:57:56 - INFO - finbert.finbert -     Num steps = 120
12/14/2025 20:57:56 - INFO - finbert.finbert -   ***** Running evaluation ***** 
12/14/2025 20:57:56 - INFO - finbert.finbert -     Num examples = 970
12/14/2025 20:57:56 - INFO - finbert.finbert -     Batch size = 32
Testing: 100%|██████████| 31/31 [00:02<00:00, 11.73it/s]
12/14/2025 20:57:58 - INFO - finbert.utils -   *** Example ***
12/14/2025 20:57:58 - INFO - finbert.utils -   guid: test-1
12/14/2025 20:57:58 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/14/2025 20:57:58 - INFO - finbert.utils -   input_ids: 10

In [13]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    print(return_val)
    return_val["Loss"] = loss
    return_val["Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [14]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [15]:
wandb_report = report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
    }

wandb.log(summary)
wandb.finish()

/var/tmp/ipykernel_158915/3181304599.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))


Loss:0.47
Accuracy:0.81

Classification Report:
{'0': {'precision': 0.706081081081081, 'recall': 0.7827715355805244, 'f1-score': 0.7424511545293073, 'support': 267.0}, '1': {'precision': 0.7017543859649122, 'recall': 0.9375, 'f1-score': 0.802675585284281, 'support': 128.0}, '2': {'precision': 0.8986083499005965, 'recall': 0.7860869565217391, 'f1-score': 0.8385899814471243, 'support': 575.0}, 'accuracy': 0.8051546391752578, 'macro avg': {'precision': 0.7688146056488633, 'recall': 0.8354528307007545, 'f1-score': 0.7945722404202376, 'support': 970.0}, 'weighted avg': {'precision': 0.8196371249948459, 'recall': 0.8051546391752578, 'f1-score': 0.807387806709082, 'support': 970.0}}


Accuracy,▁
Loss,▁
accuracy,▁
eval_num_samples,▁
eval_samples_per_s,▁
eval_wall_s,▁
model_size_mb,▁
profile_train_steps,▁
train_backward_pass_ms,▁
train_data_transfer_ms,▁
+6,...
